In [1]:
import pandas_datareader as web
import pandas as pd
from yahoo_fin import stock_info as si 
import datetime as dt


In [2]:
tickers = si.tickers_nifty50()

In [3]:
start= dt.datetime.now()-dt.timedelta(days=365)
end=dt.datetime.now()

In [4]:
nifty50_df=web.DataReader('^NSEI','yahoo',start,end)
nifty50_df['Pct Change']=nifty50_df['Adj Close'].pct_change()

In [5]:
nifty50_return=(nifty50_df['Pct Change']+1).cumprod()[-1]

In [12]:
return_list=[]
final_df=pd.DataFrame(columns=['Ticker','Latest_Price','Score','PE_Ratio','PEG_Ratio','SMA_150','SMA_200','52_Week_Low','52_Week_High',])
for ticker in tickers:
    try: 
        df=web.DataReader(ticker,'yahoo',start,end)
        df.to_csv(f'stock_data/{ticker}.csv')
        df['Pct Change']=df['Adj Close'].pct_change()
        stock_return=(df['Pct Change']+1).cumprod()[-1]
        returns_compared=round((stock_return/nifty50_return),2)
        return_list.append(returns_compared)
    except:
        print({ticker})

#best_performers=pd.DataFrame(list(zip(tickers,return_list)), columns=['Ticker','Returns Compared'])
#best_performers['Score'] = best_performers['Returns Compared'].rank(pct=True)*100
#best_performers=best_performers[best_performers['Score']>=best_performers['Score'].quantile(0.7)]

for ticker in tickers:
    try:
        df=pd.read_csv(f'stock_data/{ticker}.csv')
        moving_averages=[150,200]
        for ma in moving_averages:
            df['SMA_'+str(ma)]=round(df['Adj Close'].rolling(window=ma).mean(),2)
        latest_price=df['Adj Close'][-1]
        pe_ratio=float(si.get_quote_table(ticker)['PE Ratio(TTM)'])
        peg_ratio=float(si.get_stats_valuation(ticker)[1][4])
        moving_averages_150=df['SMA_150'][-1]
        moving_averages_200=df['SMA_200'][-1]
        low_52week=round(min(df['Low'][-(52*5):]),2)
        high_52week=round(max(df['High'][-(52*5):]),2)
        score=round(best_performers[best_performers['Ticker']==ticker]['Score'].tolist()[0])
        condition_1=latest_price>moving_averages_150>moving_averages_200
        condition_2=latest_price>=(1.3*low_52week)
        condition_3=latest_price>=(0.7*high_52week)
        condition_4=pe_ratio<40
        condition_5=peg_ratio<2
        if condition_1 and condition_2:
            final_df=final_df.append({'Ticker':ticker,
            'Latest Price':latest_price,
            'Score':score,
            'PE_Ratio':pe_ratio,
            'PEG_Ratio':peg_ratio,
            'SMA_150':moving_averages_150,
            'SMA_200':moving_averages_200,
            '52_Week_Low':low_52week,
            '52_Week_High':high_52week},ignore_index=True)
    
    except Exception as e:
        print(f"{e} for  {ticker}")

final_df.sort_values(by='Score',ascending=False)
pd.set_option('display.max_columns', 10)
print(final_df)
final_df.to_csv('final.csv')


{'MM.NS'}
-1 for  APOLLOHOSP.NS
-1 for  BAJAJ-AUTO.NS
-1 for  BAJAJFINSV.NS
-1 for  BAJFINANCE.NS
-1 for  BHARTIARTL.NS
-1 for  BRITANNIA.NS
-1 for  CIPLA.NS
-1 for  COALINDIA.NS
-1 for  HDFCLIFE.NS
-1 for  HEROMOTOCO.NS
-1 for  HINDALCO.NS
-1 for  ICICIBANK.NS
-1 for  INDUSINDBK.NS
-1 for  ITC.NS
-1 for  KOTAKBANK.NS
-1 for  LT.NS
-1 for  MARUTI.NS
[Errno 2] No such file or directory: 'stock_data/MM.NS.csv' for  MM.NS
-1 for  NESTLEIND.NS
-1 for  NTPC.NS
-1 for  ONGC.NS
-1 for  RELIANCE.NS
-1 for  SHREECEM.NS
-1 for  TATACONSUM.NS
-1 for  TATASTEEL.NS
-1 for  TCS.NS
-1 for  TECHM.NS
-1 for  TITAN.NS
-1 for  ULTRACEMCO.NS
-1 for  WIPRO.NS
Empty DataFrame
Columns: [Ticker, Latest_Price, Score, PE_Ratio, PEG_Ratio, SMA_150, SMA_200, 52_Week_Low, 52_Week_High]
Index: []
